# Getting started

This package is visualizing and processing Agilent Seahorse Extracellular Flux Analyzer data.

In the following, we'll import some sample data and give a brief overview of the current functionality.

In [ ]:
import seahorse
import pooch

# We download some sample dataset provided with a different tool (https://github.com/jamespeapen/ceas/)
url = "https://raw.githubusercontent.com/jamespeapen/ceas/05dd5af93b0fb75bdce03bf1c5ffd663cc2ee82c/inst/extdata/rep1.xlsx"
data_file = pooch.retrieve(
    url, known_hash="c584b08b0712e199502f4d7a82e5cd52582de1fe123d8706b2258640157d8108"
)

## Visualizing results

First we load the exported Seahorse data:

In [ ]:
# Read the datafile
e = seahorse.SeahorseExperiment(data_file)
e

Let's plot the oxygen consumption rate:

In [ ]:
e.plot_summary_ocr()

... and the extracellular acidification rate:

In [ ]:
e.plot_summary_ecar()

In the ECAR plot, one sample looks like an outlier, let's see which well that is to exclude it from further analysis:

In [ ]:
e.rate.query("ECAR < -100")

So it's well `E7`, let's exlude it:

In [ ]:
e.exclude_wells("E07")

For further quality control we can also look at all wells side-by-side (note how the previously excluded well is greyed out):

In [ ]:
e.small_multiples_rate()

The example file we used here does not include the raw measurements (`Raw` tab in the spreadsheet). Otherwise, we could also visualize the raw pH, O2, and temperature measurements.

## Aggregating results

We can easily access a summary of the data - the mean and standard deviations for each group and timepoint across all wells in the group:

In [ ]:
e.aggregated_rates()

If we want to use the measurements for parameter estimation for some model, the measurements can easily be transformed to the [PEtab](https://petab.readthedocs.io/en/latest/) measurements format:

In [ ]:
from seahorse.petab import to_petab

to_petab(e.aggregated_rates(), exclude_groups=["Background"])

## Further functionality

In [ ]:
# We can inspect the log:
e.log

In [ ]:
# ... access various project configuration values:
e.config()

In [ ]:
# ... see the group layout ...
group_layout = e.config()["Group Layout"].to_df()


def color_categorical(df):
    """Create a mappable color function for a DataFrame with categorical values for use with pd.DataFrame.style."""
    import matplotlib.pyplot as plt
    from matplotlib.colors import to_hex

    # unique categorical values
    keys = group_layout.melt().value.unique()
    cmap = plt.get_cmap("tab20", len(keys))
    # category => color
    cmap = {k: to_hex(cmap(i)) for i, k in enumerate(keys)}

    def color(val):
        return f"background-color: {cmap[val]}"

    return color


group_layout.style.map(color_categorical(group_layout))

In [ ]:
# see what was injected when:
e.injections